# 7. GUI creation

pyqt and pyqtgraph & pymmcore-widgets

## pyqt and pyqtgraph <a id='pyqt_and_pyqtgraph'></a>

In [4]:
from PyQt5.QtCore import QTimer
from PyQt5.QtWidgets import QApplication, QLabel, QPushButton, QWidget, QGridLayout
from PyQt5.QtGui import QFont
import pyqtgraph as pg
import numpy as np

In [5]:
### simple plot

# start application
app = pg.mkQApp()

# create window
plot_widget = pg.PlotWidget()
x = np.linspace(0, 20, 1000)
y = np.sin(x)
plot_widget.plot(x, y)
plot_widget.show()

# execute application
app.exec()


0

In [6]:
### moving sine curve ('live data acquisition')

class window(QWidget):
    def __init__(self):
        super().__init__()
        self.t = np.linspace(0, 2*np.pi, 1000)
        self.num = 0.01
        self.graph = pg.PlotWidget(self)
        self.grid = QGridLayout()
        self.timer = QTimer()
        self.grid.addWidget(self.graph, 0, 0, 1, 1)
        self.setLayout(self.grid)
        self.timer.timeout.connect(self.update)
        self.timer.start(1000) # in ms
        self.graph.show()
        self.setGeometry(0, 0, 500, 200)
        self.show()

    def update(self):
        self.graph.plotItem.clear()
        data = np.sin(self.t + self.num)
        self.graph.plotItem.plot(self.t, data)
        self.num = self.num + 0.5

app = QApplication(['auto update'])

win = window()

app.exec_()

0

In [ ]:
### display live microscopy images

class window(QWidget):
    def __init__(self):
        super().__init__()
        self.graph = pg.PlotWidget(self)
        self.grid = QGridLayout()
        self.timer = QTimer()
        self.grid.addWidget(self.graph)
        self.setLayout(self.grid)
        self.timer.timeout.connect(self.update)
        self.timer.start(1000) # in ms
        self.graph.show()
        self.show()
    
    def update(self):
        core.setConfig('LED_light', 'on')
        self.graph.plotItem.clear()
        core.snapImage()
        im = core.getImage()
        imv = pg.ImageView()
        imv.setImage(im)
        imv.show()
        #imv.close()
        core.setConfig('LED_light', 'off')

app = QApplication(['auto update'])

win = window()

app.exec_()

In [ ]:
### PyQt5 tutorial

import PyQt5.QtWidgets as qtw
import PyQt5.QtGui as qtg

class MainWindow(qtw.QWidget):
    def __init__(self):
        super().__init__()
        
        # title
        self.setWindowTitle('Microscope Automation')
        
        # layout
        self.setLayout(qtw.QHBoxLayout())
        
        # label
        label = qtw.QLabel('What`s your name?')
        # change font size of label
        label.setFont(qtg.QFont('SansSerif', 30))
        self.layout().addWidget(label)
        
        # create entry box
        entry = qtw.QLineEdit()
        entry.setObjectName('name_field')
        entry.setText('')
        self.layout().addWidget(entry)
        # create a button
        button = qtw.QPushButton('Press me!')
        
        
        self.show()

app = qtw.QApplication([])
mw = MainWindow()

app.exec_()

### pymmcore-widgets

In [ ]:
### some widgets

from qtpy.QtWidgets import QApplication
from pymmcore_widgets import PropertyBrowser, LiveButton, ExposureWidget, ImagePreview
app = QApplication([])

# create a PropertyBrowser widget. By default, this widget will use the active
# Micro-Manager core instance.

pb_widget = PropertyBrowser()
pb_widget.show()

live_btn = LiveButton()
live_btn.show()

exp_wdg = ExposureWidget()
exp_wdg.show()

img_wdg = ImagePreview()
img_wdg.show()

app.exec_()

In [ ]:
### preview widget with buttons

import os
mm_dir = 'D:\ProgramFiles\Micro-Manager-2.0'
from pymmcore_plus import CMMCorePlus

from qtpy.QtWidgets import QApplication, QGroupBox, QHBoxLayout, QVBoxLayout, QWidget

from pymmcore_widgets import (
    ChannelWidget,
    ExposureWidget,
    ImagePreview,
    LiveButton,
    SnapButton,
    ChannelGroupWidget,
)

class ImageFrame(QWidget):
    """An example widget with a snap/live button and an image preview."""

    def __init__(self):
        super().__init__()

        self.preview = ImagePreview()
        self.snap_button = SnapButton()
        self.live_button = LiveButton()
        self.exposure = ExposureWidget()
        self.channel = ChannelWidget()
        self.channel_group = ChannelGroupWidget()

        self.setLayout(QVBoxLayout())

        buttons = QGroupBox()
        buttons.setLayout(QHBoxLayout())
        buttons.layout().addWidget(self.snap_button)
        buttons.layout().addWidget(self.live_button)

        ch_exp = QWidget()
        layout = QHBoxLayout()
        layout.setContentsMargins(0, 0, 0, 0)
        ch_exp.setLayout(layout)

        ch = QGroupBox()
        ch.setTitle("Channel")
        ch.setLayout(QHBoxLayout())
        ch.layout().setContentsMargins(0, 0, 0, 0)
        ch.layout().addWidget(self.channel)
        layout.addWidget(ch)

        ch_gr = QGroupBox()
        ch_gr.setTitle("ChannelGroup")
        ch_gr.setLayout(QHBoxLayout())
        ch_gr.layout().setContentsMargins(0, 0, 0, 0)
        ch_gr.layout().addWidget(self.channel_group)
        layout.addWidget(ch_gr)

        exp = QGroupBox()
        exp.setTitle("Exposure")
        exp.setLayout(QHBoxLayout())
        exp.layout().setContentsMargins(0, 0, 0, 0)
        exp.layout().addWidget(self.exposure)
        layout.addWidget(exp)

        self.layout().addWidget(self.preview)
        self.layout().addWidget(ch_exp)
        self.layout().addWidget(buttons)


if __name__ == "__main__":
    core = CMMCorePlus()
    core.setDeviceAdapterSearchPaths([mm_dir])
    core.loadSystemConfiguration(os.path.join(mm_dir, 'MMConfig_Edge42_SOLA_ASIStage_PixelSize.cfg'))
    #core.loadSystemConfiguration()
    app = QApplication([])
    frame = ImageFrame()
    frame.show()
    #core.snap()
    app.exec_()

![Preview Window](preview_window.png "Title")

##### MainWindow with multiple widgets

In [1]:
### functions

def gauss(x, a, x0, sigma):
        return a * np.exp(-(x - x0) ** 2 / (2 * sigma ** 2))

def _software_autofocus(core, range, step_size):

        #xx, yy = core.getXYPosition()
        z = core.getZPosition()
    
        # define location and type of image saving
        writer = ImageSequenceWriter(r'C:\Users\Admin\Desktop\focus', extension=".png", overwrite=True)
    
        # acquire z-stack
        sequence = MDASequence(
            axis_order="tpgcz",
            stage_positions=[(900, 1214, z)],
            channels=[{'group': 'LED_light', 'config': 'on'}],
            z_plan={'above': range, 'below': range, 'step': step_size}
        )
    
        # run focus mda sequence
        with mda_listeners_connected(writer):
            core.mda.run(sequence)
    
        # calculate focus scores
        focus_images = glob.glob(r'C:\Users\Admin\Desktop\focus\*.png')
        focus_scores = []
        for f in focus_images:
            im = cv2.imread(rf'{f}')
            im_filtered = cv2.medianBlur(im, ksize=3)
            laplacian = cv2.Laplacian(im_filtered, ddepth=cv2.CV_64F, ksize=3)
            focus_score = laplacian.var()
            focus_scores.append(focus_score)
    
        # define x and y for fitting
        y = focus_scores
        x = np.linspace(z-range, z+range, len(y))
    
        # define gauss fit function
        mean = sum(x * y) / sum(y)
        sigma = np.sqrt(sum(y * (x - mean)**2) / sum(y))
    
        # optimize gauss fit
        popt, pcov = curve_fit(gauss, x, y, p0 = [np.max(y), mean, sigma])
    
        # calculate maximum focus score of fit function
        x_fine = np.linspace(z-range, z+range, 100)
        y_max_fit = np.max(gauss(x_fine,*popt))
        pos = np.where(y_max_fit == gauss(x_fine,*popt))
        x_max_fit = x_fine[pos]
        y_max_data = np.max(focus_scores)
        x_max_data = x[np.where(focus_scores == y_max_data)]
    
        # set optimal z position
        core.setPosition(float(x_max_fit[0]))
    
        # remove focus images
        for file in focus_images:
            os.remove(file)
        
        return x, y, x_fine, gauss(x_fine, *popt), x_max_fit, y_max_fit, x_max_data, y_max_data

### multiple widgets in main window: live view 

import os
mm_dir = 'D:\ProgramFiles\Micro-Manager-2.0'
from pymmcore_plus import CMMCorePlus
import PyQt5.QtWidgets as qtw
from qtpy.QtWidgets import QStackedWidget, QMainWindow, QAction, QApplication, QGroupBox, QWidget, QHBoxLayout, QVBoxLayout
import pymmcore_widgets as pycw
from useq import MDAEvent, MDASequence, Position
from pymmcore_plus.mda import mda_listeners_connected
from pymmcore_plus.mda.handlers import ImageSequenceWriter
import glob
import cv2
import numpy as np
from scipy.optimize import curve_fit
import pyqtgraph as pg
import matplotlib.pyplot as plt

          
class MainWindow(qtw.QWidget):
    
    def __init__(self):
        
        super().__init__()
        
        self.setLayout(qtw.QHBoxLayout())
        self.setWindowTitle('Microscope Automation')
        self.setGeometry(0, 0, 1500, 1000)
        
        ### preview ###
        preview = QGroupBox()
        preview.setLayout(qtw.QVBoxLayout())
        # define widgets #
        snap_button = pycw.SnapButton()
        live_button = pycw.LiveButton()
        image_preview = pycw.ImagePreview()
        exposure = pycw.ExposureWidget()
        channel_group = pycw.ChannelGroupWidget()
        channel = pycw.ChannelWidget()
        # snap & live buttons window #
        buttons = QGroupBox()
        buttons.setLayout(QHBoxLayout())
        buttons.layout().addWidget(snap_button)
        buttons.layout().addWidget(live_button)
        
        # channelgroup, channel & exposure window #
        ch_exp = QWidget()
        layout = QHBoxLayout()
        #layout.setContentsMargins(0, 0, 0, 0)
        ch_exp.setLayout(layout)
        ch_gr = QGroupBox()
        ch_gr.setTitle("ChannelGroup")
        ch_gr.setLayout(QHBoxLayout())
        #ch_gr.layout().setContentsMargins(0, 0, 0, 0)
        ch_gr.layout().addWidget(channel_group)
        layout.addWidget(ch_gr)
        ch = QGroupBox()
        ch.setTitle("Channel")
        ch.setLayout(QHBoxLayout())
        #ch.layout().setContentsMargins(0, 0, 0, 0)
        ch.layout().addWidget(channel)
        layout.addWidget(ch)
        
        exp = QGroupBox()
        exp.setTitle("Exposure")
        exp.setLayout(QHBoxLayout())
        #exp.layout().setContentsMargins(0, 0, 0, 0)
        exp.layout().addWidget(exposure)
        layout.addWidget(exp)
        
        # combine preview window
        preview.layout().addWidget(image_preview)
        preview.layout().addWidget(ch_exp)
        preview.layout().addWidget(buttons)
        
        ### autofocus ###
        autofocus = QWidget()
        layout = QVBoxLayout()
        autofocus.setLayout(layout)
        ## software autofocus ##
        software_autofocus = QWidget()
        layout = QHBoxLayout()
        software_autofocus.setLayout(layout)
        # range #
        af_range = QGroupBox()
        af_range.setTitle('range [µm]')
        af_range.setLayout(QHBoxLayout())
        af_range_ = qtw.QDoubleSpinBox(value = 10, maximum = 100, minimum = 5, singleStep = 5)
        af_range.layout().addWidget(af_range_)
        layout.addWidget(af_range)
        # step size #
        af_step_size = QGroupBox()
        af_step_size.setTitle('step size [µm]')
        af_step_size.setLayout(QHBoxLayout())
        af_step_size_ = qtw.QDoubleSpinBox(value = 2, maximum = 10, singleStep = 1)
        af_step_size.layout().addWidget(af_step_size_)
        layout.addWidget(af_step_size)
        # autofocus button #
        autofocus_button = QGroupBox()
        autofocus_button.setTitle('Autofocus')
        autofocus_button.setLayout(QHBoxLayout())
        autofocus_button_ = qtw.QPushButton('Autofocus')
        autofocus_button_.clicked.connect(software_autofocus(self.core))
        autofocus_button.layout().addWidget(autofocus_button_)
        layout.addWidget(autofocus_button)
        # combine software autofocus window #
        software_autofocus.layout().addWidget(af_range)
        software_autofocus.layout().addWidget(af_step_size)
        software_autofocus.layout().addWidget(autofocus_button)
        ## plot autofocus ##
        plot_autofocus = QGroupBox()
        plot_autofocus.setTitle('Plot autofocus')
        plot_autofocus.setLayout(QHBoxLayout())
        plot_autofocus_ = pg.PlotWidget()
        x = np.linspace(0, 2000, 100)
        y = np.sin(x)
        plot_autofocus_.plot(x, y, label='data')
        plot_autofocus.layout().addWidget(plot_autofocus_)
        layout.addWidget(plot_autofocus)
        # combine autofocus window #
        autofocus.layout().addWidget(software_autofocus)
        autofocus.layout().addWidget(plot_autofocus)
        
        ### XY stage widget ###
        stage_widget = pycw.StageWidget('XYStage')
        stage = QGroupBox()
        stage.setLayout(qtw.QVBoxLayout())
        stage.layout().addWidget(stage_widget)
        
        self.core = None
        self.layout().addWidget(preview)
        self.layout().addWidget(stage)
        self.layout().addWidget(autofocus)
        
        self.show()
    
        def software_autofocus(range = af_range_.value(), step_size = af_step_size_.value()):
            x, y, x_fine, y_fine, x_max_fit, y_max_fit, x_max_data, y_max_data = _software_autofocus(core, range = range, step_size = step_size)
            
            fig, ax = plt.subplots(figsize=(4, 3))
            plt.plot(x, y, label='data')
            plt.plot(x_fine, y_fine, label='fit')
            plt.plot(x_max_fit, y_max_fit, marker='X', label='max_fit', ms=10)
            plt.plot(x_max_data, y_max_data, marker='X', label='max_data', ms=10)
            plt.xlabel('z position (mu)')
            plt.ylabel('focus score')
            plt.legend()
            plt.show()
        

if __name__ == "__main__":
    core = CMMCorePlus()
    core.setDeviceAdapterSearchPaths([mm_dir])
    core.loadSystemConfiguration(os.path.join(mm_dir, 'MMConfig_Edge42_SOLA_ASIStage_PixelSize.cfg'))
    app = qtw.QApplication([])
    mw = MainWindow()
    mw.core = core
    app.exec_()

![window](window.PNG)

In [1]:
### turn LED light off

from pymmcore_plus import CMMCorePlus
mm_dir = 'D:\ProgramFiles\Micro-Manager-2.0'
import os
core = CMMCorePlus()
core.setDeviceAdapterSearchPaths([mm_dir])
core.loadSystemConfiguration(os.path.join(mm_dir, 'MMConfig_Edge42_SOLA_ASIStage_PixelSize.cfg'))
core.setConfig('LED_light', 'off')